# HW 2-1 Model Analysis Using Pytorch

In [66]:
import torchvision.models as models

# 加載 GoogLeNet 模型
model = models.googlenet(pretrained=True)
print(model)

input_shape = (3, 224, 224)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

## 2-1-1. Calculate the number of model parameters：
- include all trainable and non-trainable parameters

In [67]:
# Calculating the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  6624904


## 2-1-2. Calculate memory requirements for storing the model weights.

In [68]:
# Calculating the size of the model's parameters in bytes
param_size = sum(p.numel() * p.element_size() for p in model.parameters())
print("Total memory for parameters: ", param_size)

Total memory for parameters:  26499616


## 2-1-3. Use Torchinfo to print model architecture including the number of parameters and the output activation size of each layer

In [69]:
from torchsummary import summary

In [70]:
summary(model, input_shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
       BasicConv2d-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
       BasicConv2d-7           [-1, 64, 56, 56]               0
            Conv2d-8          [-1, 192, 56, 56]         110,592
       BatchNorm2d-9          [-1, 192, 56, 56]             384
      BasicConv2d-10          [-1, 192, 56, 56]               0
        MaxPool2d-11          [-1, 192, 28, 28]               0
           Conv2d-12           [-1, 64, 28, 28]          12,288
      BatchNorm2d-13           [-1, 64, 28, 28]             128
      BasicConv2d-14           [-1, 64,

## 2-1-4. Calculate computation requirements
- The primary focus is on calculating MACs for linear and convolutional layers, with calculations for other layers being a bonus.
- Hint: Since the MAC calculation example in the lab2-3-3 is specifically for AlexNet, it needs to be modified for use with other models.

In [74]:
import torchinfo
summary = torchinfo.summary(model, input_shape, batch_dim=0, col_names=("input_size", "output_size", "num_params", "mult_adds"), verbose=1)
total_macs = summary.total_mult_adds
print(total_macs)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
GoogLeNet                                [1, 3, 224, 224]          [1, 1000]                 --                        --
├─BasicConv2d: 1-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         --                        --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 112, 112]         9,408                     118,013,952
│    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         [1, 64, 112, 112]         128                       128
├─MaxPool2d: 1-2                         [1, 64, 112, 112]         [1, 64, 56, 56]           --                        --
├─BasicConv2d: 1-3                       [1, 64, 56, 56]           [1, 64, 56, 56]           --                        --
│    └─Conv2d: 2-3                       [1, 64, 56, 56]           [1, 64, 56, 56]           4,096                     12,845,056

In [63]:
import torch
import torchvision.models as models
import torch.nn as nn
def calculate_output_shape(input_shape, layer):
    # Calculate the output shape for Conv2d, MaxPool2d, and Linear layers
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
        kernel_size = (
            layer.kernel_size
            if isinstance(layer.kernel_size, tuple)
            else (layer.kernel_size, layer.kernel_size)
        )
        stride = (
            layer.stride
            if isinstance(layer.stride, tuple)
            else (layer.stride, layer.stride)
        )
        padding = (
            layer.padding
            if isinstance(layer.padding, tuple)
            else (layer.padding, layer.padding)
        )
        dilation = (
            layer.dilation
            if isinstance(layer.dilation, tuple)
            else (layer.dilation, layer.dilation)
        )

        if isinstance(layer, nn.MaxPool2d) and hasattr(layer, 'ceil_mode') and layer.ceil_mode:
            # 使用向上取整
            import math
            output_height = math.ceil((input_shape[1] + 2 * padding[0] - kernel_size[0]) / stride[0]) + 1
            output_width = math.ceil((input_shape[2] + 2 * padding[1] - kernel_size[1]) / stride[1]) + 1
        else:
            # 使用原公式（適用於 Conv2d 或 ceil_mode=False 的 MaxPool2d）
            output_height = (
                input_shape[1] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1
            ) // stride[0] + 1
            output_width = (
                input_shape[2] + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1
            ) // stride[1] + 1
        
        return (
            layer.out_channels if hasattr(layer, "out_channels") else input_shape[0],
            output_height,
            output_width,
        )
    elif isinstance(layer, nn.Linear):
        # For Linear layers, the output shape is simply the layer's output features
        return (layer.out_features,)
    else:
        return input_shape


def calculate_macs(layer, input_shape, output_shape):
    # Calculate MACs for Conv2d and Linear layers
    if isinstance(layer, nn.Conv2d):
        kernel_ops = (
            layer.kernel_size[0]
            * layer.kernel_size[1]
            * (layer.in_channels / layer.groups)
        )
        output_elements = output_shape[1] * output_shape[2]
        macs = int(kernel_ops * output_elements * layer.out_channels)
        return macs
    elif isinstance(layer, nn.BatchNorm2d):
        # print(f"找到 BatchNorm2d 層：{name}, MACs：{}")
        macs = int(output_shape[0] * 2)
        return macs
    elif isinstance(layer, nn.Linear):
        # For Linear layers, MACs are the product of input features and output features
        macs = int(layer.in_features * layer.out_features + (layer.out_features if layer.bias is not None else 0))
        return macs
    else:
        return 0

In [64]:
# Initial input shape
input_shape = (3, 224, 224)
total_macs = 0

# Iterate through the layers of the model
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d, nn.ReLU, nn.Linear,nn.BatchNorm2d)):
        output_shape = calculate_output_shape(input_shape, layer)
        macs = calculate_macs(layer, input_shape, output_shape)
        total_macs += macs
        if isinstance(layer, (nn.Conv2d, nn.Linear,nn.BatchNorm2d)):
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: {macs}"
            )
        elif isinstance(layer, nn.MaxPool2d):
            # Also print shape transformation for MaxPool2d layers (no MACs calculated)
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: N/A"
            )
        input_shape = output_shape  # Update the input shape for the next layer

print(f"Total MACs: {total_macs}")

Layer: conv1.conv, Type: Conv2d, Input Shape: (3, 224, 224), Output Shape: (64, 112, 112), MACs: 118013952
Layer: conv1.bn, Type: BatchNorm2d, Input Shape: (64, 112, 112), Output Shape: (64, 112, 112), MACs: 128
Layer: maxpool1, Type: MaxPool2d, Input Shape: (64, 112, 112), Output Shape: (64, 56, 56), MACs: N/A
Layer: conv2.conv, Type: Conv2d, Input Shape: (64, 56, 56), Output Shape: (64, 56, 56), MACs: 12845056
Layer: conv2.bn, Type: BatchNorm2d, Input Shape: (64, 56, 56), Output Shape: (64, 56, 56), MACs: 128
Layer: conv3.conv, Type: Conv2d, Input Shape: (64, 56, 56), Output Shape: (192, 56, 56), MACs: 346816512
Layer: conv3.bn, Type: BatchNorm2d, Input Shape: (192, 56, 56), Output Shape: (192, 56, 56), MACs: 384
Layer: maxpool2, Type: MaxPool2d, Input Shape: (192, 56, 56), Output Shape: (192, 28, 28), MACs: N/A
Layer: inception3a.branch1.conv, Type: Conv2d, Input Shape: (192, 28, 28), Output Shape: (64, 28, 28), MACs: 9633792
Layer: inception3a.branch1.bn, Type: BatchNorm2d, Input S

## 2-1-5. Use forward hooks to extract the output activations of the Conv2d layers.

In [78]:
import torch
import torchvision.models as models
activation = {}
# Define a hook function
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
model.eval()

# Register hook to each linear layer
for layer_name, layer in model.named_modules():
    if isinstance(layer, torch.nn.Conv2d):
        # Register forward hook
        layer.register_forward_hook(get_activation(layer_name))
# input_shape = (3, 224, 224)
data = torch.randn(1, 3, 224, 224)
output = model(data)

# Access the saved activations
for layer in activation:
    print(f"Activation from layer {layer}: {activation[layer].shape}")

Activation from layer conv1.conv: torch.Size([1, 64, 112, 112])
Activation from layer conv2.conv: torch.Size([1, 64, 56, 56])
Activation from layer conv3.conv: torch.Size([1, 192, 56, 56])
Activation from layer inception3a.branch1.conv: torch.Size([1, 64, 28, 28])
Activation from layer inception3a.branch2.0.conv: torch.Size([1, 96, 28, 28])
Activation from layer inception3a.branch2.1.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3a.branch3.0.conv: torch.Size([1, 16, 28, 28])
Activation from layer inception3a.branch3.1.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception3a.branch4.1.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception3b.branch1.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3b.branch2.0.conv: torch.Size([1, 128, 28, 28])
Activation from layer inception3b.branch2.1.conv: torch.Size([1, 192, 28, 28])
Activation from layer inception3b.branch3.0.conv: torch.Size([1, 32, 28, 28])
Activation from layer inception